In [1]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Batch, Data, DataLoader, InMemoryDataset
from torch_geometric.nn import MessagePassing
import torch_geometric.transforms as T

import numpy as np
import matplotlib.pyplot as plt
import os

from tqdm import tqdm, trange
from IPython.display import clear_output
from typing import Literal

torch.cuda.is_available()

True

In [2]:
from ML_functions import DEM_Dataset, Trainer, GetModel

In [4]:
file_name="N400"
model_ident = "Norm_01"
data_split=[0.85, 0.95]
pre_transform = T.Compose([T.Cartesian(False),
                           T.Distance(norm=False,cat=True)])
transform       = None
force_reload    = False
train           = False


#dataset_train     = DEM_Dataset(file_name, data_split,"train"   ,'delta', force_reload, pre_transform,transform)
#dataset_val       = DEM_Dataset(file_name, data_split,"validate",'delta', force_reload, pre_transform,transform)
dataset_test      = DEM_Dataset(file_name, data_split,"test"    ,'delta', force_reload, pre_transform,transform)

if train == True:
    model = GetModel("N400",model_ident,edge_dim=4)
    trainer = Trainer(model, dataset_train,dataset_val,
                      batch_size=32,
                      lr=0.0000001,
                      epochs=2000,
                      model_name=f"{file_name}_GCONV_Model_{model_ident}")
    #trainer.train_loop()

c:\Users\20182319\AppData\Local\anaconda3\envs\DEM-GNN2\Lib\site-packages\torch_geometric\data\dataset.py:240: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(
Processing...
18it [00:03,  5.78it/s]
 66%|██████▌   | 1168/1782 [00:17<00:00, 1376.10it/s]Done!
c:\Users\20182319\AppData\Local\anaconda3\envs\DEM-GNN2\Lib\site-packages\torch_geometric\io\fs.py:229: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([_reconstruct])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "


In [ ]:
print(dataset_train.pos.min(),dataset_train.pos.max())
print(dataset_val.pos.min(),dataset_val.pos.max())
print(dataset_test.pos.min(),dataset_test.pos.max())

print(dataset_train.x.max(dim=0)[0])
print(dataset_val.x.max(dim=0)[0])
print(dataset_test.x.max(dim=0)[0])
dataset_train.y.mean()


tensor(-1.0019) tensor(1.0152)
tensor(-0.9899) tensor(0.9682)
tensor(-0.9665) tensor(0.9697)
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
tensor([0.9893, 0.9763, 1.0002, 1.0000, 1.0000, 1.0000, 1.0000])
tensor([0.9773, 0.9903, 0.9611, 1.0000, 1.0000, 1.0000, 1.0000])


In [13]:
def FilterStart(dataset):
    idx = np.nonzero([data.time == 0 for data in dataset])
    return torch.utils.data.Subset(dataset_test,idx)[0]

#dataset_test_start = FilterStart(dataset_test)

In [14]:
from Encoding import ToPytorchData, GetLength

def GetLimits(data):
    max = [torch.max(data.x[:,i]) for i in [0, 1, 2]]
    min = [torch.min(data.x[:,i]) for i in [0, 1, 2]]
    max = torch.stack(max)
    min = torch.stack(min)
    limits = torch.stack([min,max],dim=1)
    return limits


In [15]:
class LearnedSimulator:
    def BCrollout(self):
        print("Calculating BC")
        BC_rollout = np.empty((GetLength(self.par_data),6,9))
        BC_t=np.copy(self.BC)
        for t in trange(GetLength(self.par_data)):
            BC_t[:,:3] = self.BC[:,:3]+(t+1)*self.BC[:,-3:] 
            BC_rollout[t] = BC_t
        return BC_rollout
    
    def GroundTruth_Rollout(self):
        print("Collecting Ground Truth Rollout")
        GroundTruth = np.empty(GetLength(self.par_data),dtype=object)
        for t in trange(GetLength(self.par_data)):
            data = ToPytorchData(self.par_data[t],self.BC_rollout[t],self.tol,self.topology[t])[0]
            GroundTruth[t] = data
        return GroundTruth
    
    def MLRollout(self):
        print("Calculating Learned Rollout")
        with torch.inference_mode():
            
            ML_Rollout = np.empty((self.timesteps),dtype=object)
            par_inp = self.par_data[0]
            ML_Rollout[0], MatlabTopology = ToPytorchData(par_inp,self.BC_rollout[0],self.tol)

            for t in tqdm(range(1,self.timesteps)):
                input_data = ToPytorchData(par_inp,self.BC_rollout[t],self.tol,MatlabTopology)[0]
                if self.model.mode == 'delta': 
                    #transform = T.Compose([T.Cartesian(False), T.Distance(norm=False,cat=True)])
                    transform = pre_transform
                input_data.to(self.device)

                output = self.model(transform(input_data))
                par_inp[:,:3] = par_inp[:,:3]+output[input_data.mask].cpu().numpy()
                output_data = ToPytorchData(par_inp,self.BC_rollout[t],self.tol,MatlabTopology)
                ML_Rollout[t] = output_data[0]
        return ML_Rollout
    
    def __init__(self,model,data_agr, top_agr,BC_agr,i: int,tol,timesteps=100):
        self.device = torch.device('cuda' if torch.cuda.is_available()else 'cpu')
        self.BC = BC_agr[i]
        self.par_data = data_agr[i]
        self.topology = top_agr[i]
        self.model = model.to(self.device)
        self.data_list = []
        self.timesteps = timesteps
        self.tol = tol
        self.BC_rollout = self.BCrollout()
        self.GroundTruth = self.GroundTruth_Rollout()
        self.ML_rollout = self.MLRollout()

In [16]:
from Encoding import load
def SplitData(dataset_name,data_split):
    loaded_data = load(dataset_name)
    splits=np.array(data_split)*loaded_data[0].shape[0]
    test_data = [np.split(data,splits.astype(int))[2] for data in loaded_data]
    return test_data

In [ ]:
dataset_name = "N400"
model_name = "RelPos_02"
tol = 0.5
from Plotting import PlotXYZ
for sample_idx in range (30):
    model = GetModel(dataset_name,model_name,edge_dim=3)
    AggregatedArgs = SplitData(dataset_name,data_split)
    Rollout = LearnedSimulator(model,*AggregatedArgs,i=sample_idx,tol=tol,timesteps=100)

    if dataset_name == "2Sphere": 
        PlotXYZ(Rollout,t_max=100)

In [ ]:
from Plotting import GetInternalStressRollout
stress = GetInternalStressRollout(Rollout)
torch.set_printoptions(4)
print("Stress at time 0")
print(torch.round(stress[0],decimals=8)),
print("\nStress at time 99")
print(torch.round(stress[-1],decimals=1))

In [ ]:
from Plotting import PlotContactVectorAndForce, GetAllContactpoints,AxesLimits
data = Rollout.GroundTruth[0]
BC = Rollout.BC_rollout[3]
fig,axs = PlotContactVectorAndForce(data,BC)
for ax in axs: AxesLimits(ax,BC)

In [ ]:
from Plotting import PlotGraphComparison
save = False
show = False
for t in range(20,Rollout.timesteps,20):  
    fig = PlotGraphComparison(t,Rollout,sample_idx,tol,plot_lines=True)
    if save == True: plt.savefig(f"{os.getcwd()}\\Figures\\Plots\\Graph_Sample{sample_idx}_Time{t}_Tol{str(tol)[2:]}.png",bbox_inches='tight')     
    if show == True: plt.show()
fig = PlotGraphComparison(Rollout.timesteps-1,Rollout,sample_idx,tol,plot_lines=False) 
if save == True: plt.savefig(f"{os.getcwd()}\\Figures\\Plots\\Graph_Sample{sample_idx}_Time{t}_Tol{str(tol)[2:]}.png",bbox_inches='tight')  
if show == True: plt.show()

In [ ]:
import pyvista as pv
from Plotting import MakeGIF, PlotMeshNormals


#datalist = Rollout.ML_rollout
#gifname = f"ML_2_{sample_idx}_Tol{str(tol)[2:]}"
gifname = "Test"
datalist = Rollout.GroundTruth
gifname = f"Groundtruth_2_{sample_idx}_Tol{str(tol)[2:]}"
MakeGIF(datalist,gifname,fps=8,color='lightblue',deformation=True)

#data = Rollout.ML_rollout[10]
#data = Rollout.GroundTruth[0]
#PlotMeshNormals(data)


In [ ]:
data = Rollout.ML_rollout[0]
#data = Rollout.GroundTruth[0]